# Libraries & Modules for YAMNet

In [1]:
!pip install tensorflow_io

In [2]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
import soundfile as sf

import glob
import json
import re

os.environ["CUDA_VISIBLE_DEVICES"]='-1'

"""
import warnings
warnings.filterwarnings('ignore')
"""

2024-02-19 23:19:26.042593: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 23:19:26.063382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 23:19:26.063398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 23:19:26.064181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 23:19:26.068021: I tensorflow/core/platform/cpu_feature_guar

"\nimport warnings\nwarnings.filterwarnings('ignore')\n"

# Load pretrained models & samples

In [3]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

2024-02-19 23:19:26.833620: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-19 23:19:26.833643: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: jihwan-B650M-K
2024-02-19 23:19:26.833647: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: jihwan-B650M-K
2024-02-19 23:19:26.833807: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 545.23.8
2024-02-19 23:19:26.833820: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 545.23.8
2024-02-19 23:19:26.833823: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 545.23.8


In [4]:
# Utility functions for loading audio files and making sure the sample rate is correct.

@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

# Load class mappings

# Load datasets

### Experiments (Don't run)

### Check

In [5]:
base_train_dir = "/data/GDSC_AudioPoli/testset/Training/label/"
base_valid_dir = "/data/GDSC_AudioPoli/testset/Validation/label/"
base_train_audio = '/data/GDSC_AudioPoli/testset/Training/orig/'
base_valid_audio = '/data/GDSC_AudioPoli/testset/Validation/orig/'

train_json_files = glob.glob(os.path.join(base_train_dir, '*/*.json'))
valid_json_files = glob.glob(os.path.join(base_valid_dir, '*/*.json'))
train_orig_files = glob.glob(os.path.join(base_train_audio, '*/*.wav'), recursive=True)
valid_orig_files = glob.glob(os.path.join(base_valid_audio, '*/*.wav'), recursive=True)

output_csv = '/data/GDSC_AudioPoli/AudioPoli-AI/output.csv'
output_valid_csv = '/data/GDSC_AudioPoli/AudioPoli-AI/output_valid.csv'


pd_data = pd.read_csv(output_csv)
pd_data

,filename,target,category,audio_type
0,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,226549,전기사고,voice
1,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,220721,전기사고,voice
2,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,232239,전기사고,voice
3,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,234023,전기사고,voice
4,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,222619,전기사고,voice
...,...,...,...,...
447060,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,594834,절도범죄,voice
447061,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,1238090,절도범죄,voice
447062,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,1238011,절도범죄,voice
447063,/data/GDSC_AudioPoli/testset/Training/orig/[원천...,595821,절도범죄,voice


### Drop columns with lost filenames

# Data Filtering

In [6]:
from sklearn import model_selection

output_dropna_csv = '/data/GDSC_AudioPoli/AudioPoli-AI/output_dropna.csv'
pd_data = pd.read_csv(output_dropna_csv)

my_classes = ['강제추행(성범죄)', '강도범죄', '절도범죄', '폭력범죄',
              '화재', '갇힘', '응급의료', '전기사고', '가스사고', '낙상', 
              '붕괴사고', '태풍-강풍', '지진', '도움요청', '실내', '실외'
             ]
"""
map_class_to_id = {'강제추행(성범죄)':1, '강도범죄':2, '절도범죄':3, '폭력범죄':4,
              '화재':5, '갇힘':6, '응급의료':7, '전기사고':8, '가스사고':9, '낙상':10, 
              '붕괴사고':11, '태풍-강풍':12, '지진':13, '도움요청':14, '실내':15, '실외':16}
"""
map_class_to_id = {'강제추행(성범죄)':0, '강도범죄':1, '절도범죄':2, '폭력범죄':3,
              '화재':4, '갇힘':5, '응급의료':6, '전기사고':7, '가스사고':8, '낙상':9, 
              '붕괴사고':10, '태풍-강풍':11, '지진':12, '도움요청':13, '실내':14, '실외':15}



filtered_pd = pd_data[pd_data.category.isin(my_classes)]

class_id = filtered_pd['category'].apply(lambda name: map_class_to_id[name])
filtered_pd = filtered_pd.assign(category=class_id)

# KFold (n = 5)
filtered_pd['fold'] = -1
kf = model_selection.StratifiedKFold(n_splits = 5)
for fold, (trn_, val_) in enumerate(kf.split(X=filtered_pd, y=filtered_pd['category'])):
    filtered_pd.loc[val_, 'fold'] = fold

# filtered_pd['Fold'].unique()
    
filtered_pd.head(10)
filtered_pd.count()
# filtered_pd['fold'].unique()
# filtered_pd[filtered_pd.fold == 0]

filename      441519
target        441519
category      441519
audio_type    441519
fold          441519
dtype: int64

# Audio file embeddings

In [7]:
filenames = filtered_pd['filename']
targets = filtered_pd['category']
folds = filtered_pd['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [8]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

2024-02-19 23:19:30.383321: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA


(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [9]:
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

# K-Fold


In [10]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 3)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 3)
test_ds = cached_ds.filter(lambda embedding, label, fold: fold == 4)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

# Model for embeddings

In [11]:
import gc
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import Callback

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        print("Cleaning garbage")
        tf.keras.backend.clear_session()

In [12]:
class MyModel:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()
    
    def build_model(self):
        input = layers.Input(shape=self.input_shape)
        x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(input)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(self.num_classes)(x)
        return models.Model(inputs=input, outputs=x)

    def compile_model(self):
        self.model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer="adam",
                           metrics=['accuracy'])

    def fit_model(self, train_ds, val_ds, epochs=10, batch_size=32):
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                    patience=3,
                                                    restore_best_weights=True)
        
        history = self.model.fit(train_ds,
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 validation_data=val_ds,
                                 callbacks=[ClearMemory(), callback],
                                 steps_per_epoch=folds.count()//batch_size
                                )
        
        """
        history = self.model.fit(train_ds,
                                 epochs=epochs,
                                 validation_data=val_ds,
                                 callbacks=[ClearMemory(), callback],
                                 use_multiprocessing=True,
                                 workers=20,
                                 steps_per_epoch=folds.count()//batch_size
                                )
        """
        return history
        

In [13]:
input_shape = (1024,)
num_classes = len(my_classes)
print(num_classes)
my_model = MyModel(input_shape, num_classes)
my_model.compile_model()
my_model.fit_model(train_ds, val_ds)

16
Epoch 1/10
  262/13797 [..............................] - ETA: 5:59 - loss: 3.0701 - accuracy: 0.4800

KeyboardInterrupt: 

In [ ]:
# Check for overfittings
"""
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)
"""

### Check with wav metadata

### Check whether problem due to float32 encoding

### Convert float32 to int16

### UnicodeDecodeError issue

# Testing model


In [ ]:
test_pd = filtered_pd.loc[filtered_pd['fold'] == 4]
# print(test_pd)
row = test_pd.sample(1)
filename = row['filename'].item()
print(filename)
tmp = load_wav_16k_mono(filename)

In [ ]:
# scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
scores, embeddings, spectrogram = yamnet_model(tmp)
"""
result = my_model(embeddings).numpy()

inferred_class = my_classes[result.mean(axis=0).argmax()]
print(f'The main sound is: {inferred_class}')
"""

### Pipelines for wave-input model

In [ ]:
class ReduceMeanLayer(tf.keras.layers.Layer):
  def __init__(self, axis=0, **kwargs):
    super(ReduceMeanLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.math.reduce_mean(input, axis=self.axis)

In [ ]:
saved_model_path = './dogs_and_cats_yamnet'

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
embedding_extraction_layer = hub.KerasLayer(yamnet_model_handle,
                                            trainable=False, name='yamnet')
_, embeddings_output, _ = embedding_extraction_layer(input_segment)
serving_outputs = my_model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)

In [ ]:
tf.keras.utils.plot_model(serving_model)

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
reloaded_results = reloaded_model(testing_wav_data)
cat_or_dog = my_classes[tf.math.argmax(reloaded_results)]
print(f'The main sound is: {cat_or_dog}')

In [ ]:
serving_results = reloaded_model.signatures['serving_default'](testing_wav_data)
cat_or_dog = my_classes[tf.math.argmax(serving_results['classifier'])]
print(f'The main sound is: {cat_or_dog}')

# Comparison with YAMNet models

In [ ]:
test_pd = filtered_pd.loc[filtered_pd['fold'] == 4]
# print(test_pd)
row = test_pd.sample(1)
filename = row['filename'].item()
print(filename)
waveform = load_wav_16k_mono(filename)
print(f'Waveform values: {waveform}')
_ = plt.plot(waveform)

display.Audio(waveform, rate=16000)

In [ ]:
# Run the model, check the output.
scores, embeddings, spectrogram = yamnet_model(waveform)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.math.argmax(class_scores)
inferred_class = class_names[top_class]
top_score = class_scores[top_class]
print(f'[YAMNet] The main sound is: {inferred_class} ({top_score})')

"""
reloaded_results = reloaded_model(waveform)
your_top_class = tf.math.argmax(reloaded_results)
your_inferred_class = my_classes[your_top_class]
class_probabilities = tf.nn.softmax(reloaded_results, axis=-1)
your_top_score = class_probabilities[your_top_class]
print(f'[Your model] The main sound is: {your_inferred_class} ({your_top_score})')
"""